In [1]:
import itertools

import numpy as np
import sympy

# 1.1 連立方程式

In [2]:
x, y, z = sympy.symbols("x y z")

問題1.1
$$
\begin{cases}
    x + 2y - z = 2 \\
    x + y - 4z = 3 \\
    x + 3y + 3z = 0 \\
\end{cases}
$$

In [3]:
eqs = [x + 2*y - z - 2, x + y - 4*z - 3, x + 3*y + 3*z]
res = sympy.groebner(eqs, x, y, z, order='lex')

display(res)

GroebnerBasis([x + 3, y - 2, z + 1], x, y, z, domain='ZZ', order='lex')

問題1.2
$$
\begin{cases}
    x^2 + y^2 + 4z^2 = 81 \\
    x - y + z^2 = 13 \\
    xz - 2y = 18 \\
\end{cases}
$$

In [4]:
eqs = [x**2 + y**2 + 4*z**2 - 81, x - y + z**2 - 13, x*z - 2*y - 18]
res = sympy.groebner(eqs, x, y, z, order='lex')

display(res)

GroebnerBasis([x - y + z**2 - 13, 2*y**2 + 18*y - z**4 - 4*z**3 + 4*z**2 + 16*z + 16, y*z - 2*y - z**3 + 13*z - 18, z**5 + 2*z**4 - 14*z**3 - 8*z**2 + 56*z - 32], x, y, z, domain='ZZ', order='lex')

問題1.3
$$
\begin{cases}
    x^2 + y^2 + 4z^2 = 90 \\
    x - y + z = 12 \\
    xz - 3y = 28 \\
\end{cases}
$$

In [5]:
eqs = [x**2 + y**2 + 4*z**2 - 90, x - y + z - 12, x*z - 3*y - 28]
res = sympy.groebner(eqs, x, y, z, order='lex')

display(res)

GroebnerBasis([2*x + 5*z**3 - 33*z**2 + 56*z - 18, 2*y + 5*z**3 - 33*z**2 + 54*z + 6, 5*z**4 - 48*z**3 + 155*z**2 - 180*z + 38], x, y, z, domain='ZZ', order='lex')

In [6]:
eqs = [x**2 + y**2 + 4*z**2 - 90, x - y + z - 12, x*z - 3*y - 28]
res = sympy.groebner(eqs, x, y, z, order='grlex')

display(res)

GroebnerBasis([5*z**3 - 33*z**2 + 2*y + 54*z + 6, 2*y**2 + 3*z**2 + 18*y - 2, y*z - z**2 - 3*y + 12*z - 28, x - y + z - 12], x, y, z, domain='ZZ', order='grlex')

In [7]:
eqs = [x**2 + y**2 + 4*z**2 - 90, x - y + z - 12, x*z - 3*y - 28]
res = sympy.groebner(eqs, x, y, z, order='grevlex')

display(res)

GroebnerBasis([5*z**3 - 33*z**2 + 2*y + 54*z + 6, 2*y**2 + 3*z**2 + 18*y - 2, y*z - z**2 - 3*y + 12*z - 28, x - y + z - 12], x, y, z, domain='ZZ', order='grevlex')

### 任意の連立方程式
+ 3変数で一つの変数の最大次数を設定して、そういった多項式を3つ用意した時のグレブナー基底を確認する

+ 注意: n_eq, n_termの設定に応じて、自明なグレブナー基底($<1>$から生成される多項式で、任意の多項式)しか生成されないことがある

In [8]:
# 用いる変数の数
n_var = 3

# 多項式の数
n_eq = 3

# 一つの変数の最大次数
deg = 1

# 一つの変数の次数がdeg以下の時の3変数の組み合わせ, すべて0は除くので、-1している
max_term = (deg+1)**n_var - 1

# 各多項式の項の数
n_term = 2

# 係数をランダムに決める
data_seed = 20240831
np.random.seed(data_seed)

# deg以下の次数を各変数が持つとき、すべての次数の組み合わせを使う想定であれば、(deg+1)**3の組み合わせがあるので、
coef = np.random.normal(
    size=(
        max_term,
        n_eq,
    )
)
coef = np.around(coef, decimals=2) # 係数を有理数にする

# 解の設定
solution = np.around(np.random.normal(size=n_var), decimals=1)

# 使う多項式を決める, is_useが1の項を用いる
use_rand = np.random.dirichlet(alpha=np.ones(max_term), size=n_eq)
# use_rand[:,-1] = 1 # 使いたくない項は0にする
# use_rand[:,-9:-1] = 1 # 使いたくない項は0にする
use_priority = np.argsort(use_rand).T
is_use = np.zeros((max_term, n_eq))
for k in range(n_eq):
    is_use[use_priority[:n_term,k],k] = 1

In [9]:
expres = list()
for i_var in range(n_eq):
    # expre = intercept[i_var]
    # if i_var == 0:
    #     expre = coef[0,i_var]*x
    # elif i_var == 1:
    #     expre = coef[1,i_var]*y
    # elif i_var == 2:
    #     expre = coef[2,i_var]*z
    # else:
    #     expre = 0
    expre = 0
    for ind, val in enumerate(itertools.product(range(deg+1), range(deg+1), range(deg+1))):
        if sum(val) == 0:
            # 定数項は無視する
            continue
        if is_use[ind-1,i_var] == 1:
            expre += coef[ind-1, i_var] * x**val[0]*y**val[1]*z**val[2]
        pass
    # x,y,zに解を代入して、整合性のとれた関係式を作る
    expre -= expre.subs(x, solution[0]).subs(y, solution[1]).subs(z, solution[2])
    expres.append(expre)

In [10]:
res = sympy.groebner(expres, x, y, z, order='lex')

display(res)

GroebnerBasis([1.0*x + 0.56002514142049*z**2 + 2.59280326838466*z - 0.143604651162791, 1.0*y + 2.66945317410434*z**2 + 12.3590289126336*z + 3.8121511627907, 1.0*z**4 + 9.25959595959596*z**3 + 22.6066684129563*z**2 + 5.066492954749*z - 2.02347113091508], x, y, z, domain='RR', order='lex')

In [11]:
res = sympy.groebner(expres, x, y, z, order='grlex')

display(res)

GroebnerBasis([1.0*y**2 + 4.49666666666667*y - 2.55081081081081*z - 11.8097387387387, 1.0*z**2 + 0.374608556426738*y + 4.62979797979798*z + 1.42806444397353, 1.0*x - 0.20979020979021*y - 0.943356643356643], x, y, z, domain='RR', order='grlex')

In [12]:
res = sympy.groebner(expres, x, y, z, order='grevlex')

display(res)

GroebnerBasis([1.0*y**2 + 4.49666666666667*y - 2.55081081081081*z - 11.8097387387387, 1.0*z**2 + 0.374608556426738*y + 4.62979797979798*z + 1.42806444397353, 1.0*x - 0.20979020979021*y - 0.943356643356643], x, y, z, domain='RR', order='grevlex')